In [1]:
import pandas as pd
import numpy as np
import math
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
import timeit



import matplotlib.pyplot as plt

import SMOTE
import feature_selector
import DE
import CFS
import metrics.abcd

import metrices
import measures

import sys
import traceback
import warnings
warnings.filterwarnings("ignore")

In [2]:
data_source1 = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted'
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    _dir = data_source1 + '/'
else:
    _dir = data_source1 + '\\'
projects = [f for f in listdir(_dir) if isfile(join(_dir, f))]

In [3]:
def prepare_data(path):
    df = pd.read_csv(path)
    df = df.drop(labels = ['Host','Vcs','Project','File','PL','IssueTracking'],axis=1)
    df = df.dropna()
    df = df[['TLOC', 'TNF', 'TNC', 'TND', 'LOC', 'CL', 'NStmt', 'NFunc',
       'RCC', 'MNL', 'avg_WMC', 'max_WMC', 'total_WMC', 'avg_DIT', 'max_DIT',
       'total_DIT', 'avg_RFC', 'max_RFC', 'total_RFC', 'avg_NOC', 'max_NOC',
       'total_NOC', 'avg_CBO', 'max_CBO', 'total_CBO', 'avg_DIT.1',
       'max_DIT.1', 'total_DIT.1', 'avg_NIV', 'max_NIV', 'total_NIV',
       'avg_NIM', 'max_NIM', 'total_NIM', 'avg_NOM', 'max_NOM', 'total_NOM',
       'avg_NPBM', 'max_NPBM', 'total_NPBM', 'avg_NPM', 'max_NPM', 'total_NPM',
       'avg_NPRM', 'max_NPRM', 'total_NPRM', 'avg_CC', 'max_CC', 'total_CC',
       'avg_FANIN', 'max_FANIN', 'total_FANIN', 'avg_FANOUT', 'max_FANOUT',
       'total_FANOUT', 'NRev', 'NFix', 'avg_AddedLOC', 'max_AddedLOC',
       'total_AddedLOC', 'avg_DeletedLOC', 'max_DeletedLOC',
       'total_DeletedLOC', 'avg_ModifiedLOC', 'max_ModifiedLOC',
       'total_ModifiedLOC','Buggy']]
    return df

def get_features(df):
    fs = feature_selector.featureSelector()
    df,_feature_nums,features = fs.cfs_bfs(df)
    return df,features

def apply_cfs(df):
    y = df.Buggy.values
    X = df.drop(labels = ['Buggy'],axis = 1)
    X = X.values
    selected_cols = CFS.cfs(X,y)
    cols = df.columns[[selected_cols]].tolist()
    cols.append('Buggy')
    return df[cols],cols
    
def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def tune_learner(learner, train_X, train_Y, tune_X, tune_Y, goal,loc=None,target_class=None):
    if not target_class:
        target_class = goal
    clf = learner(train_X, train_Y, tune_X, tune_Y, goal,loc)
    tuner = DE.DE_Tune_ML(clf, clf.get_param(), goal, target_class)
    return tuner.Tune()

In [4]:
final_score = {}
count = 0
for project in projects:
    try:
        start = timeit.default_timer()
        path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted/' + project
        df = prepare_data(path)
        if df.shape[0] >= 50:
            continue
        else:
            count+=1
        print(project)
        df.reset_index(drop=True,inplace=True)
        d = {'buggy': True, 'clean': False}
        df['Buggy'] = df['Buggy'].map(d)
        buggy = df[df['Buggy'] == True]
        buggy_percentage = buggy.shape[0]/df.shape[0]
        y = df.Buggy
        X = df.drop(labels = ['Buggy'],axis = 1)
        kf = StratifiedKFold(n_splits = 5)
        F = {}
        score = {}
        for i in range(5):
            for train_index, tune_index in kf.split(X, y):
                X_train, X_tune = X.iloc[train_index], X.iloc[tune_index]
                y_train, y_tune = y[train_index], y[tune_index]
                df_train = pd.concat([X_train,y_train], axis = 1)
                df_train,cols = apply_cfs(df_train)
                df_train = apply_smote(df_train)
                _df_tune_loc = X_tune.LOC
                df_tune = pd.concat([X_tune,y_tune], axis = 1)
                df_tune = df_tune[cols]
                y_train = df_train.Buggy
                X_train = df_train.drop(labels = ['Buggy'],axis = 1)
                y_tune = df_tune.Buggy
                X_tune = df_tune.drop(labels = ['Buggy'],axis = 1)
                clf = LogisticRegression()
                #clf = SVC()
                clf.fit(X_train,y_train)
                predicted = clf.predict(X_tune)
                abcd = metrices.measures(y_tune,predicted,_df_tune_loc)
                F['f1'] = [abcd.calculate_f1_score()]
                F['precision'] = [abcd.calculate_precision()]
                F['recall'] = [abcd.calculate_recall()]
                F['g-score'] = [abcd.get_g_score()]
                F['d2h'] = [abcd.calculate_d2h()]
                F['pci_20'] = [abcd.get_pci_20()]
                F['ifa'] = [abcd.get_ifa_roc()]
                F['pd'] = [abcd.get_pd()]
                F['pf'] = [abcd.get_pf()]
                _F = copy.deepcopy(F)
                print(_F)
                if 'f1' not in score.keys():
                    _F['buggy_prec'] = [buggy_percentage]
                    score = _F
                else:
                    score['f1'].append(F['f1'][0])
                    score['precision'].append(F['precision'][0])
                    score['recall'].append(F['recall'][0])
                    score['g-score'].append(F['g-score'][0])
                    score['d2h'].append(F['d2h'][0])
                    score['pci_20'].append(F['pci_20'][0])
                    score['ifa'].append(F['ifa'][0])
                    score['pd'].append(F['pd'][0])
                    score['pf'].append(F['pf'][0])
                    score['buggy_prec'].append(buggy_percentage)
            final_score[project] = score
        stop = timeit.default_timer() 
        print("Model training time: ", stop - start)   
    except Exception as e:
        print(e)
        continue

lcdata.csv
n_splits=5 cannot be greater than the number of members in each class.
llcon.csv
float division by zero
gpsee.csv
{'f1': [0.67], 'precision': [0.67], 'recall': [0.67], 'g-score': [0.69], 'd2h': [0.39], 'pci_20': [0.67], 'ifa': [58.5], 'pd': [0.67], 'pf': [0.2]}
{'f1': [0.57], 'precision': [0.5], 'recall': [0.67], 'g-score': [0.63], 'd2h': [0.6], 'pci_20': [0.25], 'ifa': [78.5], 'pd': [0.67], 'pf': [0.5]}
float division by zero
tauruss.csv
float division by zero
makumba.csv
{'f1': [0.44], 'precision': [1.0], 'recall': [0.29], 'g-score': [0.34], 'd2h': [0.71], 'pci_20': [0.5], 'ifa': [37.5], 'pd': [0.29], 'pf': [0.0]}
{'f1': [0.73], 'precision': [0.8], 'recall': [0.67], 'g-score': [0.67], 'd2h': [0.47], 'pci_20': [0.4], 'ifa': [75.5], 'pd': [0.67], 'pf': [0.33]}
{'f1': [0.77], 'precision': [0.71], 'recall': [0.83], 'g-score': [0.64], 'd2h': [0.69], 'pci_20': [0.29], 'ifa': [218.5], 'pd': [0.83], 'pf': [0.67]}
{'f1': [0.83], 'precision': [0.83], 'recall': [0.83], 'g-score': [0.

KeyboardInterrupt: 

In [ ]:
final_score

In [ ]:
with open('data/1385/1385_LR_smote_cfs_test_other.pkl', 'wb') as handle:
    pickle.dump(final_score, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
df = pd.read_pickle('data/1385/1385_LR_smote_cfs_test.pkl')
results = []
for project in df.keys():
    results.append([project,np.median(df[project]['f1']),
                   np.median(df[project]['precision']),
                   np.median(df[project]['recall']),
                   np.median(df[project]['g-score']),
                   np.median(df[project]['d2h']),
                   np.median(df[project]['pci_20']),
                   np.median(df[project]['ifa']),
                   np.median(df[project]['pd']),
                   np.median(df[project]['pf']),
                   np.median(df[project]['buggy_prec'])])

In [ ]:
results = []
for project in df.keys():
    results.append([project,np.median(df[project]['f1']),
                   np.median(df[project]['precision']),
                   np.median(df[project]['recall']),
                   np.median(df[project]['g-score']),
                   np.median(df[project]['d2h']),
                   np.median(df[project]['pci_20']),
                   np.median(df[project]['ifa']),
                   np.median(df[project]['pd']),
                   np.median(df[project]['pf']),
                   np.median(df[project]['buggy_prec'])])

In [ ]:
results_df = pd.DataFrame(results,columns = ['project','f1','precision','recall','g-score','d2h','pci_20','ifa','pd','pf','buggyness'])


In [ ]:
results_df.to_csv('data/1385/1385_LR_smote_cfs_test.csv')

In [ ]:
score = np.array([.68,.54,.77,.66,.66,.73,.5,.77,.62,.58,.7,.46,.73,.66,.66,.76,.54,.56,.66,.66,.71,.29,.69,.58,.54])
print(np.median(score))
plt.boxplot(score)